In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [64]:
url_inicial = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2016'
url_root = 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2016'
r=requests.get(url_inicial)

In [65]:
soup = BeautifulSoup(r.text, 'html.parser')

In [66]:
box = soup.find('div', class_='sumarios decada-2016')
volumen=box.findAll('div', class_='sumario row')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-30-num-4-S0187533717X00034',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-30-num-3-S0187533717X00022',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-30-num-2-S0187533716X00040',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-sumario-vol-30-num-1-S0187533716X00039']

In [67]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1 = [urljoin (url_root, i) for i in vol1]
    vol2 = [urljoin (url_root, i) for i in vol2]
    vol2+=vol1
    vol2+=vol1

In [68]:
vol2

['https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-nuevas-evidencias-patogenia-preeclampsia-importancia-S0187533717300432',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-estudio-comparativo-sobre-efectividad-del-S018753371730016X',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-utilidad-un-metodo-que-determina-S0187533717300146',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-funcion-cardiaca-fetal-preeclampticas-tratadas-S0187533717300122',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-complicaciones-accesos-vasculares-centrales-recien-S0187533717300109',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-patrones-susceptibilidad-gramnegativos-aislamientos-nosocomiales-S0187533717300092',
 'https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-articulo-c

In [69]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='items') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [70]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.elsevier.es/es-revista-perinatologia-reproduccion-humana-144-numeros-anteriores-anio-2016


In [71]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

68

In [72]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [73]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Perinatología y Reproducción Humana'
    a='III. Medicina y Ciencias de la Salud'
    tem='Perinatología, Neonatología, Obstetricia, Reproducción Humana'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('span', class_='elsevierStyleSections').get_text(strip=True)
        content_book['Resumen']=resu.replace("Objetivo","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='logo').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('div', class_='right').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [74]:
list_scraper=list_scraper[0:74]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [75]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Nuevas evidencias de la patogenia de la preecl...,A pesar de que la preeclampsia constituye una ...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
1,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Estudio comparativo sobre la efectividad del h...,"IntroducciónActualmente, el ambiente hospitala...",,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
2,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Utilidad de un método que determina simultánea...,IntroducciónLa leche materna se ha recomendado...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
3,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Función cardiaca fetal en preeclámpticas trata...,"IntroducciónLa preeclampsia, una de las causas...",,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
4,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Complicaciones de los accesos vasculares centr...,IntroducciónEl acceso vascular constituye en l...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
...,...,...,...,...,...,...,...,...
63,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Exposición a oxitocina sintética en el periodo...,IntroducciónLa Organización Mundial de Salud d...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
64,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Psicosis puerperal. Panorama general sobre su ...,IntroducciónLas primeras observaciones registr...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
65,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Monitoreo de morbilidad materna extrema(near m...,AntecedentesEl registro y análisis de las muer...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
66,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Los trastornos de ansiedad durante la transici...,MenopausiaLa menopausia es la interrupción def...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus


In [76]:
df_catalogo.to_csv('Revista095.03.csv', index=False)

In [77]:
df=df_catalogo.drop(df_catalogo.index[[52,60]])
df

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Nuevas evidencias de la patogenia de la preecl...,A pesar de que la preeclampsia constituye una ...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
1,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Estudio comparativo sobre la efectividad del h...,"IntroducciónActualmente, el ambiente hospitala...",,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
2,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Utilidad de un método que determina simultánea...,IntroducciónLa leche materna se ha recomendado...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
3,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Función cardiaca fetal en preeclámpticas trata...,"IntroducciónLa preeclampsia, una de las causas...",,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
4,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Complicaciones de los accesos vasculares centr...,IntroducciónEl acceso vascular constituye en l...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
...,...,...,...,...,...,...,...,...
63,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Exposición a oxitocina sintética en el periodo...,IntroducciónLa Organización Mundial de Salud d...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
64,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Psicosis puerperal. Panorama general sobre su ...,IntroducciónLas primeras observaciones registr...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
65,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Monitoreo de morbilidad materna extrema(near m...,AntecedentesEl registro y análisis de las muer...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus
66,Perinatología y Reproducción Humana,III. Medicina y Ciencias de la Salud,"Perinatología, Neonatología, Obstetricia, Repr...",Los trastornos de ansiedad durante la transici...,MenopausiaLa menopausia es la interrupción def...,,https://www.elsevier.es/es-revista-perinatolog...,https://www.elsevier.es/es-coronavirus


In [79]:
df.to_csv('Revista095.03.csv', index=False)